In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split


df = pd.read_csv("summarization_data_newsroom.csv")
df = df.drop(columns=['density_bin'])

traindf, testdf = train_test_split(df, test_size=0.2)

train = Dataset.from_pandas(traindf)
test = Dataset.from_pandas(testdf)


dataset = DatasetDict()

dataset['train'] = train
dataset['test'] = test

/opt/anaconda3/envs/prjenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer

checkpoint = "t5-small" #model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [3]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [4]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [5]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [6]:
import evaluate

rouge = evaluate.load("rouge")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 import evaluate                                                                              │
│   2                                                                                              │
│ ❱ 3 rouge = evaluate.load("rouge")                                                               │
│   4                                                                                              │
│                                                                                                  │
│ /opt/anaconda3/envs/prjenv/lib/python3.11/site-packages/evaluate/loading.py:731 in load          │
│                                                                                                  │
│   728 │   │   `evaluate.EvaluationModule`                                                        │
│   729 │   """                                                                                    │
│   730 │   download_mode = DownloadMode(download_mode or DownloadMode.REUSE_DATASET_IF_EXISTS)    │
│ ❱ 731 │   evaluation_module = evaluation_module_factory(                                         │
│   732 │   │   path, module_type=module_type, revision=revision, download_config=download_confi   │
│   733 │   )                                                                                      │
│   734 │   evaluation_cls = import_main_class(evaluation_module.module_path)                      │
│                                                                                                  │
│ /opt/anaconda3/envs/prjenv/lib/python3.11/site-packages/evaluate/loading.py:680 in               │
│ evaluation_module_factory                                                                        │
│                                                                                                  │
│   677 │   │   │   │   except Exception as e2:  # noqa: if it's not in the cache, then it doesn   │
│   678 │   │   │   │   │   pass                                                                   │
│   679 │   │   │   if not isinstance(e1, (ConnectionError, FileNotFoundError)):                   │
│ ❱ 680 │   │   │   │   raise e1 from None                                                         │
│   681 │   │   │   raise FileNotFoundError(                                                       │
│   682 │   │   │   │   f"Couldn't find a module script at {relative_to_absolute_path(combined_p   │
│   683 │   │   │   │   f"Module '{path}' doesn't exist on the Hugging Face Hub either."           │
│                                                                                                  │
│ /opt/anaconda3/envs/prjenv/lib/python3.11/site-packages/evaluate/loading.py:639 in               │
│ evaluation_module_factory                                                                        │
│                                                                                                  │
│   636 │   │   │   │   │   │   │   │   download_config=download_config,                           │
│   637 │   │   │   │   │   │   │   │   download_mode=download_mode,                               │
│   638 │   │   │   │   │   │   │   │   dynamic_modules_path=dynamic_modules_path,                 │
│ ❱ 639 │   │   │   │   │   │   │   ).get_module()                                                 │
│   640 │   │   │   │   │   │   except ConnectionError:                                            │
│   641 │   │   │   │   │   │   │   pass                                                           │
│   642 │   │   │   │   │   raise FileNotFoundError                                                │
│                                                            

In [7]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [8]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [9]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 training_args = Seq2SeqTrainingArguments(                                                   │
│    2 │   output_dir="./results",                                                                 │
│    3 │   evaluation_strategy="epoch",                                                            │
│    4 │   learning_rate=2e-5,                                                                     │
│ in __init__:116                                                                                  │
│                                                                                                  │
│ /opt/anaconda3/envs/prjenv/lib/python3.11/site-packages/transformers/training_args.py:1344 in    │
│ __post_init__                                                                                    │
│                                                                                                  │
│   1341 │   │   │   and (get_xla_device_type(self.device) != "GPU")                               │
│   1342 │   │   │   and (self.fp16 or self.fp16_full_eval)                                        │
│   1343 │   │   ):                                                                                │
│ ❱ 1344 │   │   │   raise ValueError(                                                             │
│   1345 │   │   │   │   "FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half  │
│   1346 │   │   │   │   " (`--fp16_full_eval`) can only be used on CUDA devices."                 │
│   1347 │   │   │   )                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation 
(`--fp16_full_eval`) can only be used on CUDA devices.